In [12]:
import tensorflow as tf
from PIL import Image
import os
import numpy as np
from search_videos_in_directory import search_videos
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
input_direc = input("Enter the absolute path of the input directory: ")
output_direc = input("Enter the absolute path of the output directory: ")
try:
    os.mkdir(output_direc)
except FileExistsError:
    pass
search_videos(input_direc, output_direc)

Enter the absolute path of the input directory: /home/debanik/downloaded_videos
Enter the absolute path of the output directory: /home/debanik/PycharmProjects/Detecting-Deepfakes/Face_images_Deepfakes_only__1
Enter the number of images to be generated per video file: 2


KeyboardInterrupt: 

In [3]:
for root, dirs, files in os.walk(output_direc):
    for file in files:

        try:
            im = Image.open(root + '/' + str(file))
            im.verify()
        except:
            print(str(file))
            os.remove(root + '/' + str(file))

In [ ]:
input_direc = '/home/debanik/downloaded_videos'
output_direc = '/home/debanik/PycharmProjects/Detecting-Deepfakes/Face_images_Deepfakes_only_1'

In [11]:
# create generator
datagen = ImageDataGenerator(samplewise_center=True, 
                             samplewise_std_normalization=True, 
                             rotation_range=10, 
                             horizontal_flip = True, 
                             validation_split=0.2)
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory(output_direc, class_mode='binary', batch_size=64,subset='training')
val_it = datagen.flow_from_directory(output_direc, class_mode='binary', batch_size=64,subset='validation')
# # confirm the iterator works
# batchX, batchy = train_it.next()
# print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [6]:
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2

In [7]:
x = Input(shape = (256,256, 3))
#ImageNet weights
model_mn = MobileNetV2(input_shape = (224,224,3),alpha=1.3,include_top=False)

#Use the generated model 
output_mn = model_mn(x)

#Add the fully-connected layers 
y = Flatten()(output_mn)
y = Dense(1000, activation = 'relu')(y)
y = Dense(1000, activation = 'relu')(y)
y = Dropout(0.5)(y)
y = Dense(1, activation='sigmoid')(y)


W0625 15:16:30.515887 140238720796480 deprecation.py:506] From /home/debanik/PycharmProjects/Detecting-Deepfakes/venv_cpu/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
model = Model(inputs = x, outputs = y)

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
mobilenetv2_1.30_224 (Model) multiple                  3766048   
_________________________________________________________________
flatten (Flatten)            (None, 106496)            0         
_________________________________________________________________
dense (Dense)                (None, 1000)              106497000 
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout (Dropout)            (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1001  

In [10]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model.fit_generator(train_it,validation_data=val_it, epochs = 10)

W0625 15:16:41.981884 140238720796480 deprecation.py:323] From /home/debanik/PycharmProjects/Detecting-Deepfakes/venv_cpu/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10


ValueError: Empty training data.